In [2]:
from scipy.special import expit
from rbm import RBM
from sampler import VanillaSampler, PartitionedSampler
from trainer import VanillaTrainier
from performance import Result
import numpy as np
import datasets, performance, plotter, mnist, pickle, rbm, os, logging, sampler
from sklearn.linear_model import Perceptron

logger = logging.getLogger()
# Set the logging level to logging.DEBUG 
logger.setLevel(logging.INFO)


%matplotlib inline
# make one 2 bit image, both visibles on
v = np.ones(2)

# lets make our hiddens, 2 hidden units
h_a = np.zeros(2)
h_b = np.zeros(2)
h_a[0] = 1
h_a[1] = 0

h_b[0] = 0
h_b[1] = 1

# Set up our weights matrix (|h| , |v|), with perfect weights.
w_a = np.ones((2, 2))
w_b = np.ones((2, 2))
w_a[0][1] = -1
w_a[1][0] = -1
w_b[0][1] = -1
w_b[1][0] = -1

A = RBM(2,2,1)
B = RBM(2,2,1)
# manually set our RBM's
A.hidden = h_a
B.hidden = h_b
A.visible = v
B.visible = v
A.weights = w_a
B.weights = w_b
A.hidden_bias = 0.5
B.hidden_bias = 0.5
A.visible_bias = 0.5
B.visible_bias = 0.5

In [3]:
def count_matrix_for_runs(runs, model, visible):
    van_sampler = VanillaSampler(model)
    counts = {}
    for i in range(runs):

        vis = np.where(van_sampler.hidden_to_visible(van_sampler.visible_to_hidden(visible)) >= 0.5, 1, 0)
        key = np.array_str(vis)
        if key in counts:
            counts[key] = counts[key] + 1
        else:
            counts[key] = 0
    for key in counts:
        counts[key] = counts[key]/runs * 100
    
    return counts

In [5]:
# run vanilla sampling 10,000 times, getting the percentage of reconstructions for A and then B
print(count_matrix_for_runs(10000, A, v))
print(count_matrix_for_runs(10000, B, v))

{'[1 0]': 23.46, '[1 1]': 52.43, '[0 1]': 24.08}
{'[1 0]': 23.810000000000002, '[1 1]': 53.410000000000004, '[0 1]': 22.75}


In [4]:
def weighted_sum_into_vis(vis_idx, w,h):
    return np.dot(w.T, h)

def visible_to_hidden(visible):
    np.dot(w.T,visibe)


In [5]:
def __bernouli_flip__(weighted_sum):
        p = expit(weighted_sum) > np.random.rand(*weighted_sum.shape)
        return np.where(p, 1, 0)
    
def calc_correction(hidden_a, hidden_b, weights_a, weights_b):        
    phi_a = np.dot(hidden_a, weights_a)
    phi_b = np.dot(hidden_b, weights_b)

    on_weights_a = (hidden_a * weights_a)
    off_weights_a = (1 - hidden_a) * weights_a

    on_weights_b =  (hidden_b * weights_b)
    off_weights_b = (1 - hidden_b) * weights_b
    
    j_off_a = phi_a - on_weights_a
    j_off_b = phi_b - on_weights_b
    j_on_a = phi_a + off_weights_a
    j_on_b = phi_b + off_weights_b
    
    correction_a = np.log(expit(j_off_a))  - np.log(expit((j_off_a + phi_b))) + np.log(expit((j_on_a + phi_b))) - np.log(expit(j_on_a))
    correction_b = np.log(expit(j_off_b))  - np.log(expit((j_off_b + phi_a))) + np.log(expit((j_on_b + phi_a))) - np.log(expit(j_on_b))
    #print("calc'd\nc_a{}\t for h_a{}\nc_b{}\t for h_b{}".format(correction_a, h_a, correction_b, h_b))
    return correction_a, correction_b



def run_partitioned_samples(hidden_a, hidden_b, weights_a, weights_b, visible,num_samples, vis_bias_a, vis_bias_b, hid_bias_a, hid_bias_b, c = calc_correction):
    for epoch in range(num_samples):

        correction_a, correction_b = c(hidden_a, hidden_b, weights_a, weights_b)
        """
        Apply the correction to the weighted sum into the hiddens
        """
        psi_a = ( weights_a.T * (visible + correction_a)).sum(1) + hid_bias_a
        psi_b = ( weights_b.T * (visible + correction_b)).sum(1) + hid_bias_b
#         print(psi_a, psi_b)
        
        # now, do we turn on he hiddens? Bernoulli sample to decide
        hidden_a = np.where(psi_a > np.random.rand(*psi_a.shape),1,0)  #__bernouli_flip__(psi_a)
        hidden_b = np.where(psi_b > np.random.rand(*psi_b.shape),1,0)  #__bernouli_flip__(psi_b)

    return hidden_a, hidden_b

def runit(times, sample_times,h_a, h_b, w_a, w_b,v):
    wins = []
    for i in range(times):
        h_a, h_b = run_partitioned_samples(h_a,h_b, w_a,w_b, v, sample_times, 0, 0,0,0)
        wins.append((h_a != h_b).all())
        
    return wins

Recall that $$ \psi_{j} = \sum_i W_{ji} (v_i  + \sigma_{ij}^A - \sigma_{ij}^{AB}) $$
And therefore we would expect:
$$ \psi_{j=0} = \sum_i W_{0i} (v_i  + \sigma_{i0}^A - \sigma_{i0}^{AB}) $$

In [5]:
h_a, h_b = run_partitioned_samples(h_a,h_b, w_a,w_b, v, 500, 0.5, 0.5,0.5,0.5)
print("h_a{} h_b{}".format(h_a, h_b))

[ 0.5466772  0.4533228] [ 0.4533228  0.5466772]
[ 0.4144412  0.5855588] [ 0.4144412  0.5855588]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.4533228  0.5466772] [ 0.5466772  0.4533228]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.4533228  0.5466772] [ 0.5466772  0.4533228]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.4533228  0.5466772] [ 0.5466772  0.4533228]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5466772  0.4533228] [ 0.4533228  0.5466772]
[ 0.4533228  0.5466772] [ 0.5466772  0.4533228]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5855588  0.4144412] [ 0.5855588  0.4144412]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.4533228  0.5466772] [ 0.5466772  0.4533228]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.4533228  0.5466772] [ 0.5466772  0.4533228]
[ 0.5  0.5] [ 0.

In [8]:
# do 200 runs of partitioned sampling with each run finding the hidden state with 10,000 gibbs alternations
wins = runit(200,10000, h_a, h_b, w_a,w_b,v)
win_count = np.array(wins).sum()
lose_count = len(wins) - win_count
print("wins:{} losses:{}".format(win_count, lose_count))
# damn we wanted to wins to be greater than the losses.

[ 0.  0.] [ 0.  0.]
[  2.22044605e-16   0.00000000e+00] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[ 0.0466772 -0.0466772] [-0.0466772  0.0466772]
[ 0.  0.] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[-0.0466772  0.0466772] [ 0.0466772 -0.0466772]
[ 0.  0.] [ 0.  0.]
[ 0.0466772 -0.0466772] [-0.0466772  0.0466772]
[ 0.  0.] [ 0.  0.]
[ 0.0855588 -0.0855588] [ 0.0855588 -0.0855588]
[ 0.0466772 -0.0466772] [-0.0466772  0.0466772]
[  2.22044605e-16   0.00000000e+00] [ 0.  0.]
[  0.00000000e+00   2.22044605e-16] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[  2.22044605e-16   0.00000000e+00] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[ 0.  0.] [ 0.  0.]
[  0.00000000e+00   2.22044605e-16] [ 0.  0.]
[-0.0855588  0.0855588] [-0.0855588  0.0855588]
[ 0.0466772 -0.0466772] [-0.0466772  0.0466772]
[-0.0466772  0.0466772] [ 0.0466772 -0.0466772]
[  0.00000000e+00   2.22044605e-16] [ 0.  0.]
[ 0.0855588 -0.0855588] [ 0.0855588 -0.0855588]
[ 0.  0.] [ 0.  0.]
[ 0.0466772 -0.0466772] [-0.0466772  0.0466772]
[-0.

KeyboardInterrupt: 

In [9]:
c_a, c_b = calc_correction(np.array([1,1]), np.array([0,1]), w_a, w_b)

$$ \psi_{j} = \sum_{i} (W_{ji}v_i + C_{ji}) $$

In [67]:
col_hid_a = h_a.reshape(2,1)
col_hid_b = h_b.reshape(2,1)
phi_a = np.dot(w_a, h_a) 
phi_b = np.dot(w_b, h_b) 
phi_a = phi_a - (col_hid_a)
phi_b = phi_b - (col_hid_b)
sig_A = phi_a + w_a/2
# print(sig_A)
sig_B = phi_b + w_b/2
epsilon_a = np.dot(w_b,h_b)
epsilon_b = np.dot(w_a,h_a)
sig_AB = sig_A + epsilon_a
sig_BA = sig_B + epsilon_b
c_a = expit(sig_A) - expit(sig_AB)
c_b = expit(sig_B) - expit(sig_BA)
print(phi_a)

print(phi_a - (w_a * h_a_r))

[ 1. -1.]
[[ 0.  0.]
 [ 1. -1.]]


In [73]:
def approx_correction(h_a, h_b, w_a, w_b):
    col_hid_a = h_a.reshape(2,1)
    col_hid_b = h_b.reshape(2,1)
    phi_a = np.dot(w_a, h_a) - (w_a * col_hid_a)
    phi_b = np.dot(w_b, h_b) - (w_b * col_hid_b)
    sig_A = phi_a + w_a/2
    sig_B = phi_b + w_b/2
    epsilon_a = np.dot(w_b,h_b)
    epsilon_b = np.dot(w_a,h_a)
    sig_AB = sig_A + epsilon_a
    sig_BA = sig_B + epsilon_b
    c_a = expit(sig_A) - expit(sig_AB)
    c_b = expit(sig_B) - expit(sig_BA)
    return c_a, c_b



In [87]:
def p_hid(h_a, h_b, w_a, w_b, v_bias = 0.0, h_bias = 0.0, c = approx_correction):
    c_a, c_b = c(h_a, h_b, w_a, w_b)
    psi_a = (w_a * (v + c_a)).sum(1) # of course this isn't really the correction it's more of an ammendent (? word)
    psi_b = (w_b * (v + c_b)).sum(1) 
    return expit(psi_a),expit(psi_b)

So I will run this approximated correction and check relevant $\psi$ values (without any bias' applies)

In [88]:
def calculate_result_dict(w_a, w_b, c = approx_correction):
    results = {}
    base_key = "h_a{} h_b{}"
    for i_a in range(2):
        for j_a in range(2):
            for i_b in range(2):
                for j_b in range(2):
                    hidden_a = np.array([j_a, i_a]) 
                    hidden_b = np.array([j_b, i_b])
                    results[base_key.format(hidden_a, hidden_b)] = p_hid(hidden_a, hidden_b, w_a, w_b,c= c)
    return results       

def probs_for_result_dict(results, hidd_counts):
    tot = 0
    for i in hidd_counts:
        tot += hidd_counts[i]
    for key in hidd_counts:
        print("{} {}% of the time. {} times out of {}.\n\tRespective Probabilities:{}\n\t".format(key, hidd_counts[key]/tot,  hidd_counts[key], tot,results[key] ))
        
    print("tot = {}".format(tot))

In [92]:
r = calculate_result_dict(w_a, w_b)

In [91]:
h_a = np.array([1,0])
h_b = np.array([0,1])

p_hid(h_a, h_b, w_a, w_b)[0]

array([ 0.62006811,  0.37993189])

In [93]:
r

{'h_a[0 0] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[0 1]': (array([ 0.62006811,  0.40366186]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[1 0]': (array([ 0.40366186,  0.62006811]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.5,  0.5])),
 'h_a[0 1] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.62006811,  0.40366186])),
 'h_a[0 1] h_b[0 1]': (array([ 0.59633814,  0.40366186]),
  array([ 0.59633814,  0.40366186])),
 'h_a[0 1] h_b[1 0]': (array([ 0.37993189,  0.62006811]),
  array([ 0.62006811,  0.37993189])),
 'h_a[0 1] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.59633814,  0.37993189])),
 'h_a[1 0] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.40366186,  0.62006811])),
 'h_a[1 0] h_b[0 1]': (array([ 0.62006811,  0.37993189]),
  array([ 0.37993189,  0.62006811])),
 'h_a[1 0] h_b[1 0]': (array([ 0.40366186,  0.59633814]),
  array([ 0.40366186,  0.59633814])),
 'h_a[1 0] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.37993189,  0.59633814])),
 'h_

In [28]:
def runit_approx(times, sample_times,h_a, h_b, w_a, w_b,v, c = approx_correction):

    res = {}
    base_key = "h_a{} h_b{}"
    for i in range(times):
        h_a, h_b = run_partitioned_samples(h_a,h_b, w_a,w_b, v, sample_times, 0.5, 0.5,0.5,0.5, c)
        key = base_key.format(h_a, h_b)
        if key not in res:
            res[key] = 1
        else:
            res[key] = res[key] + 1 
    return res

In [69]:
hiddens = runit_approx(10000,100, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.0492% of the time. 492 times out of 10000.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.039% of the time. 390 times out of 10000.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.048% of the time. 480 times out of 10000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.0451% of the time. 451 times out of 10000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.2096% of the time. 2096 times out of 10000.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.0504% of the time. 504 times out of 10000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.0234% of the time. 234 times out of 10000.
	Respective Probabilities:(array([ 0

In [67]:
hiddens = runit_approx(10000,200, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.0469% of the time. 469 times out of 10000.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.0438% of the time. 438 times out of 10000.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.0499% of the time. 499 times out of 10000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.0472% of the time. 472 times out of 10000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.216% of the time. 2160 times out of 10000.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.0471% of the time. 471 times out of 10000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.0225% of the time. 225 times out of 10000.
	Respective Probabilities:(array([ 

In [68]:
hiddens = runit_approx(1000,500, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.054% of the time. 54 times out of 1000.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.036% of the time. 36 times out of 1000.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.048% of the time. 48 times out of 1000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.036% of the time. 36 times out of 1000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.214% of the time. 214 times out of 1000.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.052% of the time. 52 times out of 1000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.027% of the time. 27 times out of 1000.
	Respective Probabilities:(array([ 0.5,  0.5]), array([

In [70]:
hiddens = runit_approx(100,1000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[0 0] h_b[0 0] 0.01% of the time. 1 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0 1] h_b[1 1] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.2% of the time. 20 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0 0] h_b[1 1] 0.

In [71]:
hiddens = runit_approx(100,2000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.29% of the time. 29 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0

In [72]:
hiddens = runit_approx(100,5000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.08% of the time. 8 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.17% of the time. 17 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1

In [73]:
hiddens = runit_approx(100,10000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.05% of the time. 5 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.28% of the time. 28 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.05% of the time. 5 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1

In [74]:
hiddens = runit_approx(100,15000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.01% of the time. 1 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.05% of the time. 5 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.18% of the time. 18 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.05% of the time. 5 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0

In [75]:
hiddens = runit_approx(100,20000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.23% of the time. 23 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 0] h_b[1 1] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1

In [82]:
hiddens = runit_approx(100,1000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

h_a[0 0] h_b[0 1] 0.01% of the time. 1 times out of 100.
	Respective Probabilities:(array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 0] 0.02% of the time. 2 times out of 100.
	Respective Probabilities:(array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 0] 0.12% of the time. 12 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732]))
	
h_a[0 1] h_b[1 1] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[0 1] h_b[1 0] 0.2% of the time. 20 times out of 100.
	Respective Probabilities:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
	
h_a[0 1] h_b[0 0] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971]))
	
h_a[1 1] h_b[1 1] 0.01% of the time. 1 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0

In [85]:
hiddens = runit_approx(1,1000, h_a, h_b, w_a,w_b,v)
probs_for_result_dict(r, hiddens)

[ 0.5  0.5] [ 0.98983732  0.10976971]
[ 0.28686531  0.89023029] [ 0.28686531  0.89023029]
[ 0.89023029  0.28686531] [ 0.89023029  0.28686531]
[ 0.28686531  0.89023029] [ 0.28686531  0.89023029]
[ 0.89023029  0.28686531] [ 0.89023029  0.28686531]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.89023029  0.28686531] [ 0.89023029  0.28686531]
[ 0.5  0.5] [ 0.10976971  0.98983732]
[ 0.89023029  0.28686531] [ 0.89023029  0.28686531]
[ 0.5  0.5] [ 0.10976971  0.98983732]
[ 0.98983732  0.10976971] [ 0.5  0.5]
[ 0.5  0.5] [ 0.10976971  0.98983732]
[ 0.89023029  0.01016268] [ 0.01016268  0.89023029]
[ 0.5  0.5] [ 0.10976971  0.98983732]
[ 0.89023029  0.28686531] [ 0.89023029  0.28686531]
[ 0.10976971  0.98983732] [ 0.5  0.5]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.5  0.5] [ 0.10976971  0.98983732]
[ 0.98983732  0.10976971] [ 0.5  0.5]
[ 0.5  0.5] [ 0.10976971  0.98983732]
[ 0.89023029  0.01016268] [ 0.01016268  0.89023029]
[ 0.89023029  0.01016268] [ 0.01016268  0.89023029]
[ 0.5  0.5] [ 0.5  0.5]
[ 0.01016268  0.89023029] [ 

In [1]:
def approx_correction_dbl(h_a, h_b, w_a, w_b):
    phi_a = np.dot(w_a, h_a)
    phi_b = np.dot(w_b, h_b)
    sig_A = phi_a + w_a*2
    sig_B = phi_b + w_b*2
    epsilon_a = np.dot(w_b,h_b)
    epsilon_b = np.dot(w_a,h_a)
    sig_AB = sig_A + epsilon_a
    sig_BA = sig_B + epsilon_b
    c_a = expit(sig_A) - expit(sig_AB)
    c_b = expit(sig_B) - expit(sig_BA)
    return c_a, c_b


In [29]:
r2= calculate_result_dict(w_a, w_b, c=approx_correction_dbl)
hiddens2 = runit_approx(100,1000, h_a, h_b, w_a,w_b,v,c= approx_correction_dbl)
probs_for_result_dict(r2, hiddens2)

h_a[1 0] h_b[0 0] 0.03% of the time. 3 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.3564459,  0.799477 ]))
	
h_a[1 1] h_b[1 1] 0.09% of the time. 9 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[0 0] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0 1] h_b[1 1] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.799477 ,  0.3564459]))
	
h_a[1 0] h_b[1 1] 0.06% of the time. 6 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.3564459,  0.799477 ]))
	
h_a[0 1] h_b[1 0] 0.09% of the time. 9 times out of 100.
	Respective Probabilities:(array([ 0.200523 ,  0.6435541]), array([ 0.6435541,  0.200523 ]))
	
h_a[1 0] h_b[0 1] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.6435541,  0.200523 ]), array([ 0.200523 ,  0.6435541]))
	
h_a[0 0] h_b[1 0] 0.

In [30]:
r2

{'h_a[0 0] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[0 1]': (array([ 0.799477 ,  0.3564459]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[1 0]': (array([ 0.3564459,  0.799477 ]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.5,  0.5])),
 'h_a[0 1] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.799477 ,  0.3564459])),
 'h_a[0 1] h_b[0 1]': (array([ 0.96211716,  0.44112067]),
  array([ 0.96211716,  0.44112067])),
 'h_a[0 1] h_b[1 0]': (array([ 0.200523 ,  0.6435541]),
  array([ 0.6435541,  0.200523 ])),
 'h_a[0 1] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.799477 ,  0.3564459])),
 'h_a[1 0] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.3564459,  0.799477 ])),
 'h_a[1 0] h_b[0 1]': (array([ 0.6435541,  0.200523 ]),
  array([ 0.200523 ,  0.6435541])),
 'h_a[1 0] h_b[1 0]': (array([ 0.44112067,  0.96211716]),
  array([ 0.44112067,  0.96211716])),
 'h_a[1 0] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.3564459,  0.799477 ])),
 'h_a[1 1] h_b[0 0]': (a

In [31]:
r

{'h_a[0 0] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[0 1]': (array([ 0.98983732,  0.10976971]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[1 0]': (array([ 0.10976971,  0.98983732]), array([ 0.5,  0.5])),
 'h_a[0 0] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.5,  0.5])),
 'h_a[0 1] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971])),
 'h_a[0 1] h_b[0 1]': (array([ 0.89023029,  0.28686531]),
  array([ 0.89023029,  0.28686531])),
 'h_a[0 1] h_b[1 0]': (array([ 0.01016268,  0.89023029]),
  array([ 0.89023029,  0.01016268])),
 'h_a[0 1] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.98983732,  0.10976971])),
 'h_a[1 0] h_b[0 0]': (array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732])),
 'h_a[1 0] h_b[0 1]': (array([ 0.89023029,  0.01016268]),
  array([ 0.01016268,  0.89023029])),
 'h_a[1 0] h_b[1 0]': (array([ 0.28686531,  0.89023029]),
  array([ 0.28686531,  0.89023029])),
 'h_a[1 0] h_b[1 1]': (array([ 0.5,  0.5]), array([ 0.10976971,  0.98983732])),
 'h_

In [34]:
key1 = 'h_a[1 0] h_b[0 1]'
key2 = 'h_a[0 1] h_b[1 0]'
print("weights/2\n\t{}:{}\n\t{}:{}\nweights*2\n\t{}:{}\n\t{}:{}".format(key1, r[key1],key2, r[key2], key1, r2[key1],key2, r2[key2]))

weights/2
	h_a[1 0] h_b[0 1]:(array([ 0.89023029,  0.01016268]), array([ 0.01016268,  0.89023029]))
	h_a[0 1] h_b[1 0]:(array([ 0.01016268,  0.89023029]), array([ 0.89023029,  0.01016268]))
weights*2
	h_a[1 0] h_b[0 1]:(array([ 0.6435541,  0.200523 ]), array([ 0.200523 ,  0.6435541]))
	h_a[0 1] h_b[1 0]:(array([ 0.200523 ,  0.6435541]), array([ 0.6435541,  0.200523 ]))


In [35]:
r3= calculate_result_dict(w_a, w_b, c=calc_correction)
hiddens3 = runit_approx(100,1000, h_a, h_b, w_a,w_b,v,c= calc_correction)
probs_for_result_dict(r3, hiddens3)

h_a[1 0] h_b[0 0] 0.05% of the time. 5 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[1 1] 0.01% of the time. 1 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1 1] h_b[0 0] 0.05% of the time. 5 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0 1] h_b[1 1] 0.04% of the time. 4 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[1 1] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[0 0] h_b[0 1] 0.09% of the time. 9 times out of 100.
	Respective Probabilities:(array([ 0.5,  0.5]), array([ 0.5,  0.5]))
	
h_a[1 0] h_b[0 1] 0.07% of the time. 7 times out of 100.
	Respective Probabilities:(array([ 0.5466772,  0.4533228]), array([ 0.4533228,  0.5466772]))
	
h_a[0 0] h_b[1 0] 0.09% of the time. 9 times out of 100.
	Respective Probabiliti

In [14]:
c_a, c_b = psi(np.array([1,0]),np.array([0,1]), w_a,w_b,v_bias=0, h_bias=0)
c_a

array([ 0.39023029, -0.48983732])